# Notebook for Clustering and Annotating using `scvi` - noninf only

**Created by :** Srivalli Kolla

**Created on :** 27 May, 2025

**Modified on :** 27 May, 2025

**University of Würzburg**

Env : scanpy (Python 3.12.2)

# Importing Packages

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import datetime
import os
import torch
import scvi
import plotnine as p
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()

plt.rcParams['figure.dpi'] = 300  
plt.rcParams['savefig.dpi'] = 300

timestamp = datetime.datetime.now().strftime("%d_%m_%y")

/tmp/ipykernel_1327052/4146390281.py:2: FutureWarning: Use `print_header` instead


# Data import

In [3]:
cd './Github/Nuclear_hashing_Mag_beads_2025/'

/home/gruengroup/srivalli/Github/Nuclear_hashing_Mag_beads_2025


In [4]:
adata = sc.read_h5ad('./data/demultiplexed_cellbender_singlets_scrublet_doublet_removal_magnetic_beads_22_05_25.h5ad')
adata

AnnData object with n_obs × n_vars = 3914 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'ident', 'Sample', 'Sample-ID', 'Mouse-ID', 'Sex', 'Group', 'Nuclei Purification Method after Hashing', 'assigned_hashtag', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'assigned_hashtag_colors', 'log1p'
    layers: 'cpm_normalization', 'raw_counts'

In [5]:
adata = adata[adata.obs['Group']=='noninf'].copy()
adata

AnnData object with n_obs × n_vars = 1474 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'ident', 'Sample', 'Sample-ID', 'Mouse-ID', 'Sex', 'Group', 'Nuclei Purification Method after Hashing', 'assigned_hashtag', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'assigned_hashtag_colors', 'log1p'
    layers: 'cpm_normalization', 'raw_counts'

In [6]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_HTO,nFeature_HTO,HTO_maxID,HTO_secondID,HTO_margin,HTO_classification,HTO_classification.global,...,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,percent_chrY,XIST-counts,XIST-percentage,gender_check_cov,doublet_scores,predicted_doublets
TCTAGCTTCCTCTAGT-1,SeuratProject,98198.0,10855,2320.0,8,TotalSeqB7,TotalSeqB4,1.028045,TotalSeqB7,Singlet,...,3083,3.490637,611,0.691787,0.007926,209,0.236634,Female,0.079872,0
CAATCCTTCCGGTTAA-1,SeuratProject,90072.0,10007,1629.0,8,TotalSeqB7,TotalSeqB1,1.096554,TotalSeqB7,Singlet,...,2051,2.539403,370,0.458108,0.011143,77,0.095336,Female,0.164948,0
TAGGTGGAGGTGCATC-1,SeuratProject,88193.0,10866,2921.0,8,TotalSeqB5,TotalSeqB4,1.678000,TotalSeqB5,Singlet,...,831,1.056083,266,0.338048,0.000000,214,0.271964,Female,0.136531,0
TCTCTATGTTGACCCA-1,SeuratProject,72892.0,9886,2095.0,8,TotalSeqB7,TotalSeqB4,0.924476,TotalSeqB7,Singlet,...,1504,2.386733,210,0.333254,0.006348,139,0.220582,Female,0.087838,0
CATTTGAGTCATCGGC-1,SeuratProject,71269.0,9715,2093.0,8,TotalSeqB7,TotalSeqB6,1.253093,TotalSeqB7,Singlet,...,2499,4.062622,311,0.505592,0.000000,135,0.219469,Female,0.164948,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGTCACCAACATTAG-1,SeuratProject,14628.0,4102,1548.0,8,TotalSeqB7,TotalSeqB5,0.434041,TotalSeqB7,Singlet,...,0,0.000000,5,0.104603,0.000000,25,0.523013,Female,0.079872,0
GGAAATTGTCCATACG-1,SeuratProject,13967.0,3790,2208.0,8,TotalSeqB5,TotalSeqB3,0.281725,TotalSeqB5,Singlet,...,0,0.000000,4,0.096061,0.000000,24,0.576369,Female,0.038674,0
GTAGTAAAGGATGCCT-1,SeuratProject,14096.0,4069,2076.0,8,TotalSeqB5,TotalSeqB6,0.450568,TotalSeqB5,Singlet,...,0,0.000000,6,0.138793,0.000000,10,0.231321,Female,0.052632,0
ACCACCTCAATTAACG-1,SeuratProject,14117.0,3773,2051.0,8,TotalSeqB5,TotalSeqB3,0.402708,TotalSeqB5,Singlet,...,0,0.000000,1,0.020458,0.000000,5,0.102291,Female,0.045187,0


#### Check if data is raw or Normalized

In [7]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

In [8]:
print(X_is_raw(adata))

False


In [9]:
adata.X= adata.layers['raw_counts']
print(X_is_raw(adata))

True


#### HVG 

In [10]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 8000,
    layer = "raw_counts",
    batch_key = 'assigned_hashtag',
    subset = True
)
adata

extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 1474 × 8000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'ident', 'Sample', 'Sample-ID', 'Mouse-ID', 'Sex', 'Group', 'Nuclei Purification Method after Hashing', 'assigned_hashtag', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'assigned_hashtag_colors', 'log1p', 'hvg'
    layers: 'cpm_normalization', 'raw_counts'

In [11]:
adata.X= adata.layers['cpm_normalization']
print(X_is_raw(adata))

False


# SCVI

## Data preparation

In [12]:
scvi.model.SCVI.setup_anndata(adata, layer="raw_counts", batch_key='assigned_hashtag')
adata

AnnData object with n_obs × n_vars = 1474 × 8000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'ident', 'Sample', 'Sample-ID', 'Mouse-ID', 'Sex', 'Group', 'Nuclei Purification Method after Hashing', 'assigned_hashtag', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'doublet_scores', 'predicted_doublets', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'assigned_hashtag_colors', 'log1p', 'hvg', '_scvi_uuid', '_scvi_manager_uuid'
    layers: 'cpm_normalization', 'raw_counts'

## Building the model

In [13]:
model_scvi = scvi.model.SCVI.setup_anndata(adata, 
                              layer = 'raw_counts',
                              categorical_covariate_keys = ['assigned_hashtag'])
model_scvi

In [14]:
model_scvi = scvi.model.SCVI(adata, 
                             dispersion = 'gene-batch', 
                             gene_likelihood = 'nb')
model_scvi

SCVI model with the following parameters: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene-batch, gene_likelihood: nb, 
latent_distribution: normal.
Training status: Not Trained
Model's adata is minified?: False

In [15]:
model_scvi.view_anndata_setup()

Anndata setup with scvi-tools version 1.3.1.post1.

Setup via `SCVI.setup_anndata` with arguments:

{
│   'layer': 'raw_counts',
│   'batch_key': None,
│   'labels_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': ['assigned_hashtag'],
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │   1   │
│         n_cells          │ 1474  │
│ n_extra_categorical_covs │   1   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   1   │
│          n_vars          │ 8000  │
└──────────────────────────┴───────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │         adata.layers['raw_counts']         │
│         batch          │          adata.obs['_scvi_batch']          │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
│         labels         │         adata.obs['_scvi_labels']          │
└────────────────────────┴────────────────────────────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                  extra_categorical_covs State Registry                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃        Source Location        ┃   Categories    ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['assigned_hashtag'] │ noninf_MB_Hash5 │          0          │
│                               │ noninf_MB_Hash7 │          1          │
│                               │                 │                     │
└───────────────────────────────┴─────────────────┴─────────────────────┘

## Training the model

In [16]:
max_epochs_scvi = np.min([round((20000 / adata.n_obs) * 400), 400])
max_epochs_scvi

np.int64(400)

In [17]:
model_scvi.train(200, 
                 check_val_every_n_epoch = 1, 
                 enable_progress_bar = True
                 )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training:   0%|          | 0/200 [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


### Evaluate model performance by la Svensson

In [ ]:
history_df = (
    model_scvi.history['elbo_train'].astype(float)
    .join(model_scvi.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

plot = (
    p.ggplot(history_df.query('epoch > 0'), p.aes(x='epoch', y='value', color='variable'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual(values={'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
    + p.theme(figure_size=(12, 6))
)

plot


## Extracting the embedding

In [18]:
adata.obsm["X_scVI"] = model_scvi.get_latent_representation()

## Calculate a batch-corrected UMAP

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata)
adata

## Data Visualization

In [ ]:
features = [
    'Sex', 'Group', 'HTO_classification', 'assigned_hashtag',
    'Nuclei Purification Method after Hashing', 'total_counts',
    'n_genes_by_counts', 'pct_counts_mt', 'pct_counts_ribo', 'doublet_scores']

sc.pl.umap(
        adata,
        color=features,
        frameon=False,
        layer='cpm_normalization',
        cmap='RdYlBu_r',legend_loc='right margin' 
    )


In [ ]:
sc.pl.umap(adata,color = ['Myh6','Dcn','Col1a1','Pecam1','Cdh5','Myh11','Acta2'],frameon= False, cmap = 'RdYlBu_r')

### Leiden Clustering

In [ ]:
sc.tl.leiden(adata, resolution=1, key_added="leiden_1")

In [ ]:
sc.tl.leiden(adata, resolution=0.1, key_added="leiden_0.1")

In [ ]:
sc.tl.leiden(adata, resolution=0.2, key_added="leiden_0.2")

In [ ]:
sc.tl.leiden(adata, resolution=0.3, key_added="leiden_0.3")

In [ ]:
sc.tl.leiden(adata, resolution=0.5, key_added="leiden_0.5")

In [ ]:
sc.pl.umap(adata, color=["leiden_0.1","leiden_0.2","leiden_0.3","leiden_0.5","leiden_1"],frameon= False,legend_loc="on data",)

### Scoring Marker Genes

In [28]:
marker_genes = {
    'B cells': ['Cd19', 'Ly6d', 'Ptprc'],
    'Cardiomyocytes (CM)': [
        'Angpt1', 'Btk', 'MbNkx2-5Actc1Mdh2', 'Myh6', 'Myh7',
        'Slit2', 'Slit3', 'Tnnt2', 'Ttn', 'Xirp2','Nppa','Nppb','Myh7'
    ],
    'Dendritic cells (DC)': [
        'Ccr7', 'Cd209a', 'H2-Ab1', 'Ifitm1', 'Itgae', 'Itgax',
        'Ly6c2SiglechCd8a', 'Mgl2', 'Ptprc'
    ],
    'Erythrocyte': ['Gypa'],
    'Fibroblasts (FB)': [
        'Ackr2', 'Actc2', 'Cd109', 'Cd302', 'Cd9', 'Ccl14', 'Ccl2', 'Cmah',
        'Col11a1', 'Cxcl3', 'Dkk2', 'Dkk3', 'Duox1', 'Fgl2', 'Il1rap1',
        'Isg15', 'Mki67', 'Msln', 'Nrxn1', 'Pdgfra', 'PdgfralowActc2',
        'Pdgfrb', 'Postn', 'Saa3', 'Thbs4', 'Wif1','Actc2', 'Pdgfra', 'Ptprc'
    ],
    'ILC2': ['Bmp7', 'Cd13', 'Gata3', 'Il2ra', 'Il5', 'Il7r', 'Kit', 'Ptprc'],
    'MAIT-like': [
        'Cd3e', 'Il17a', 'Il23r', 'Kcnk1', 'Mmp25', 'Ptprc', 'Pxdc1', 'Rorc', 'Trac','Cd3e', 'Il17a', 'Il23r', 'Mmp25', 'Ptprc', 'Rorc', 'Trac'
    ],
    'Mast cells': ['Cma1', 'Cpa3', 'Kit', 'Ptprc'],
    'Macrophages': [
        'Cxcl13', 'Cxcl3', 'Fn1', 'Gpnmb', 'H2-Ab1', 'Il1b', 'Il10', 'Isg15',
        'Itgam', 'Ly6c2', 'Lyve1', 'Ptprc', 'Retnla', 'Spp1', 'Timd4', 'Trem2'
    ],
    'Neutrophils': [
        'Cxcl2', 'Il1b', 'Ly6g', 'Ptprc', 'S100a8', 'S100a9'
    ],
    'NK cells': [
        'Camk1d', 'Cd3e', 'Cd44', 'Gzma', 'Klra5', 'Klra8', 'Ptprc', 'Xcr1'
    ],
    'Schwann cells': [
        'Camk1d', 'Cd63', 'Cxcl10', 'Hexb', 'Ifi27l2a', 'Isg15', 'Lgals3',
        'Mgp', 'Mt2', 'Pcna', 'Plp1', 'Prnp'
    ],
    'T cells': [
        'Cd3e', 'Cd4', 'Cd8a', 'Cd44', 'Dusp10', 'Foxp3', 'H2-Ab1', 'Ifit1',
        'Ifit3', 'Ifngr2', 'Il1r1', 'Il2ra', 'Isg15', 'Itgam', 'Itgav',
        'Klrg1', 'Lamc1', 'Ptprc', 'Rorc', 'Sell', 'Trac', 'Trcg-C1',
        'Trdc'
    ],
    'Endothelial cells': [
        'Areg', 'Apq1', 'Camk1d', 'Ccl21a', 'Cdh13', 'Cdh5', 'Col4a1',
        'Cxcl12', 'Fbln5', 'Fscn1', 'Hexb', 'H2-Ab1', 'Ift122', 'Ifit3',
        'Isg15', 'Kit', 'Lyve1', 'Mgp', 'Mmrn1', 'Prnp', 'Rbp7', 'Sema3g',
        'Tcf15', 'Timp4', 'Vcam1', 'Vwf', 'Wnt'
    ],
    'Vascular - Epicardial-derived cells': ['Msln', 'Mgp', 'Pdgfra', 'Pdgfrb'],
    'Pericytes': [
        'Acta2', 'Dkk2', 'Ifit3', 'Isg15', 'Kcnj8', 'Mgp', 'Msln', 'Pdgfra', 'Pdgfrb'
    ],
    'Smooth muscle cells (SMC)': [
        'Acta2', 'Fos', 'Jun', 'Myh11', 'Ogn', 'Rgs5', 'Ryr2', 'S100a6', 'Slc38a11', 'Tcap'
    ],
    'Not annotated': []
}

In [29]:
marker_genes_in_data = {}
for ct, markers in marker_genes.items():
    markers_found = []
    for marker in markers:
        if marker in adata.var.index:
            markers_found.append(marker)
    marker_genes_in_data[ct] = markers_found

In [ ]:
for cell_type, genes in marker_genes.items():
    valid_genes = [g for g in genes if g in adata.var_names]
    if len(valid_genes) >= 2:
        sc.tl.score_genes(adata, gene_list=valid_genes, score_name=f"{cell_type}_score", use_raw=False)

In [ ]:
score_cols = [f"{ct}_score" for ct in marker_genes.keys()]
existing_scores = [col for col in score_cols if col in adata.obs.columns]
sc.pl.umap(adata, color=existing_scores, cmap='RdYlBu_r', vmin=0, frameon= False)

In [ ]:
cluster_means = (
    adata.obs.groupby("leiden_1")[existing_scores]
    .mean()
)

In [33]:
top_scores = cluster_means.idxmax(axis=1).str.replace("_score", "")

In [34]:
adata.obs["celltype_marker_genes"] = adata.obs["leiden_1"].map(top_scores)

In [ ]:
sc.pl.umap(adata, color = "celltype_marker_genes",frameon= False)

## Differentially Expressed Genes based Annotation

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_1', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata,n_genes = 5)

In [ ]:
sc.get.rank_genes_groups_df(adata, group=None).groupby("group").head(10)

In [39]:
marker_dfs = []

for i in adata.obs['leiden_1'].cat.categories:
    df = sc.get.rank_genes_groups_df(adata, group=i).head(10).copy()
    df["cluster"] = i  
    marker_dfs.append(df)

all_markers = pd.concat(marker_dfs)

all_markers.to_csv("./data/top10_de_clean_noninf_scvi.csv", index=False)

In [51]:
cluster_annotations = {
    "0":  "Ventricular cardiomyocytes",
    "1":  "Ventricular/atrial cardiomyocytes",
    "2":  "Endothelial cells / Hematopoietic (immune) cells",
    "3":  "Atrial cardiomyocytes",
    "4":  "Conduction system cells",
    "5":  "Fibroblasts / Stromal cells",
    "6":  "Conduction system cells",
    "7":  "Cardiac fibroblasts",
    "8":  "Mitochondrial-rich or stressed cardiomyocytes",
    "9":  "Uncharacterized/unknown",
    "10": "Vascular endothelial cells",
    "11": "Lymphocytes",
    "12": "B-cells / Stromal",
    "13": "Fibroblasts / Perivascular cells",
    "14": "Pericytes / Vascular smooth muscle cells",
    "15": "Stressed/natriuretic cardiomyocytes"
}

adata.obs['celltype_de_genes'] = adata.obs['leiden_1'].map(cluster_annotations)

In [ ]:
sc.pl.umap(adata, color='celltype_de_genes',frameon = False)

# Comapring Annotation methods

In [ ]:
pd.crosstab(adata.obs["celltype_de_genes"], adata.obs["celltype_marker_genes"])

In [ ]:
sb.heatmap(
    pd.crosstab(adata.obs["celltype_de_genes"], adata.obs["celltype_marker_genes"]),
    annot=True,
    fmt='d',
    annot_kws={"size": 6}
)

In [55]:
def combine_annotations(de, score):
    if de == score:
        return de
    elif score in ["Cardiomyocyte", "Endothelial", "B cell", "T cell", "NK", "Fibroblast"]:
        return score + " (scored)"
    else:
        return de + " (DE)"

adata.obs["final_annotation"] = [
    combine_annotations(de, score)
    for de, score in zip(adata.obs["celltype_de_genes"], adata.obs["celltype_marker_genes"])
]

In [ ]:
sc.pl.umap(adata, color="final_annotation", title="Final Cell Type Annotation", frameon= False)